# Impact of Challenger Prize Money Increases on Number of Players Earning at Least $X

The number of players earning at least a certain amount can be manipulated by changing the amount of prize money player's receive at various tournament levels. The following illustrates by what factor does Challenger level prize money need to be increased for a certain number of players to earn at least the given amount of money.

In [3]:
import pandas as pd
import datetime
import json

In [4]:
END_YEAR = datetime.datetime.now().year - 1
CHALLENGER_PRIZE_MONEY_FACTOR = 3.5
MIN_EARNINGS = 75000

In [5]:
def clean_tournaments_string(tournaments_string):
    return tournaments_string.replace("'", '"')

In [6]:
experimental_data = []
total_additional_prize_money = 0

df = pd.read_csv(f"../data/players/combined/{END_YEAR}.gz")
for i, _ in df.iterrows():
    player_prize_money = df.loc[i, "total_prize_money"]

    clean_singles = clean_tournaments_string(df.loc[i, "singles_tournaments"])
    singles_tournaments = json.loads(clean_singles[clean_singles.find("["):])

    for singles_tournament in singles_tournaments:
        if singles_tournament["tournament_type"] == "CH":
            tournament_prize_money = int(singles_tournament["prize_money"])
            additional_prize_money = tournament_prize_money * (CHALLENGER_PRIZE_MONEY_FACTOR - 1)

            player_prize_money += additional_prize_money
            total_additional_prize_money += additional_prize_money

    clean_doubles = clean_tournaments_string(df.loc[i, "doubles_tournaments"])
    doubles_tournaments = json.loads(clean_doubles[clean_doubles.find("["):])
    for doubles_tournament in doubles_tournaments:
        if doubles_tournament["tournament_type"] == "CH":
            tournament_prize_money = int(doubles_tournament["prize_money"])
            additional_prize_money = tournament_prize_money * (CHALLENGER_PRIZE_MONEY_FACTOR - 1)

            player_prize_money += additional_prize_money
            total_additional_prize_money += additional_prize_money

    experimental_data.append({
        "player_id": df.loc[i, "player_id"],
        "type": df.loc[i, "type"],
        "total_prize_money": player_prize_money,
        "singles_tournaments": singles_tournaments,
        "doubles_tournaments": doubles_tournaments
    })

experimental_df = pd.DataFrame(experimental_data)
print("Total additional prize money:", total_additional_prize_money)


Total additional prize money: 44988040.0


In [7]:
data = []
total = 0
singles = 0
doubles = 0

for i, row in experimental_df.iterrows():
    if row["total_prize_money"] >= MIN_EARNINGS:
        total += 1
        if row["type"] == "singles":
            singles += 1
        elif row["type"] == "doubles":
            doubles += 1

data.append({
    "year": END_YEAR,
    "total": total,
    "singles": singles,
    "doubles": doubles,
})

results = pd.DataFrame(data)

In [8]:
results

,year,total,singles,doubles
0,2023,510,342,168
